In [1]:
import pandas as pd
data = pd.read_csv('./bank-additional/bank-additional-full.csv')
print(data.shape)

(41188, 21)


In [2]:
data[:5]

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [1]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session(
    boto_session=boto3.session.Session(profile_name="jerome-admin")
)


In [12]:
prefix = 'sagemaker/DEMO-smprocessing/input'
input_data = sagemaker_session.upload_data(
    bucket='s3-data-science',
    path='./bank-additional/bank-additional-full.csv', 
    key_prefix=prefix
)
print(input_data)

s3://s3-data-science/sagemaker/DEMO-smprocessing/input/bank-additional-full.csv


In [7]:
from sagemaker.sklearn.processing import SKLearnProcessor


boto3.setup_default_session(profile_name='jerome-admin')
iam = boto3.client('iam')
execution_role = iam.get_role(RoleName='SageMakerExecution')['Role']['Arn']
                    
sklearn_processor = SKLearnProcessor(
framework_version='0.23-1',
role=execution_role,
instance_type='ml.t3.medium',
instance_count=1)

In [6]:
from sagemaker.processing import ProcessingInput,ProcessingOutput

input_data='s3://s3-data-science/sagemaker/DEMO-smprocessing/input/bank-additional-full.csv'

sklearn_processor.run(
code='preprocessing.py',
inputs=[
   ProcessingInput(
   source=input_data,
   destination='/opt/ml/processing/input')
],
outputs=[
   ProcessingOutput(
     source='/opt/ml/processing/train',
     output_name='train_data'),
     ProcessingOutput(
     source='/opt/ml/processing/test',
     output_name='test_data'
   )
],
arguments=['--train-test-split-ratio', '0.2']
)


Job Name:  sagemaker-scikit-learn-2021-04-27-10-39-23-932
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://s3-data-science/sagemaker/DEMO-smprocessing/input/bank-additional-full.csv', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-2-629874083750/sagemaker-scikit-learn-2021-04-27-10-39-23-932/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'train_data', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-2-629874083750/sagemaker-scikit-learn-2021-04-27-10-39-23-932/output/train_data', 'LocalPath': '/opt/ml/processing/train', 'S3UploadMode': 'EndOfJob'}}, {'Ou